In [1]:
# IN PROGRESS...
# IE DOES NOT WORK STAY BACK
#
# THE STELLAR INTERPRETATION OF QUANTUM MECHANICS
# THANKS TO ETTORE MAJORANA. VERSION 5.0
#
# Now with free will. See settings below...
#
# Choose:
n=4
# Choose n where n is the dimensionality of the randomly 
# generated quantum system, which is in a pure state.
# This is visualized by n-1 white stars in the sky.
# The pure state is decomposed into tensor pieces,
# one for each prime factor of n. So a 12d pure state
# factors into 3 pieces: 2x2, 2x2, and 3x3 mixed states. We 
# take primes in order of magnitude; obviously this is an
# arbitrary choice. Each piece is assigned a randomly
# chosen color. Each nxn density matrix leads to n constellations
# of n-1 stars in the sky, all of the same color, with the sizes
# aka heights of the stars in each constellation given by the eigenvalue 
# corresponding to the eigenvector of the density matrix.
# For a prime dimensional system, the pure state is composed with
# its conjugate to form a density matrix whose eigenvalues/eigenvectors
# are constellated and displayed alongside the pure stars.

import vpython
import numpy as np
import scipy.linalg
import math
import mpmath
import qutip
import random
import sys

# Returns a list of the prime factors
# of the integer n ordered by magnitude.
def factorize(n):
    if n == 0 or n == 1:
        return [n]
    primfac = []
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)
            n //= d
        d += 1
    if n > 1:
       primfac.append(n)
    return primfac

# Stereographically projects an XYZ cartesian coordinate on 
# the Riemann sphere to a point on the complex plane.
def flatten(x, y, z):
    if z == 1:
        return Inf 
    else:
        return complex(x/(1-z), y/(1-z))

# Reverse stereographically projects a point on the complex plane
# to an XYZ cartesian coordinate on the Riemann sphere.
def rollup(z):
    x = z.real
    y = z.imag
    return (2*x)/(1+(x**2)+(y**2)), \
            (2*y)/(1+(x**2)+(y**2)), \
            (-1+(x**2)+(y**2))/(1+(x**2)+(y**2))
        
# Takes a complex vector, interprets its ordered components as
# the coefficients of a polynomial in one variable, and solves for
# the roots, giving an unordered set of complex numbers. 
# These complex points are reverse stereographically projected onto 
# the Riemann sphere, and the XYZ cartesian coordinates of these points 
# are returned. When there's a ZeroDivisionError aka we're at infinity,
# we map to the point (0,0,1) aka the North Pole.
def constellate(v):
    try:
        roots = mpmath.polyroots(v)
    except:
         return [[0,0,1] for i in range(len(v))]
    return [[float(a) for a in rollup(root)] for root in roots]

# Takes a matrix, in this case, a density matrix, and finds its spectrum.
# Two lists are returned: of eigenvalues and the corresponding constellations of each eigenvector,
# signifying the constellations at what "height" or "scale" between heaven and earth.
def emanate(m):
    l, v = np.linalg.eig(m)
    return l, [constellate(v[i]) for i in range(len(v))]

# Takes a complex vector, in this case, the pure state of a quantum system,
# and shatters it into pieces depending on the prime factorization of
# the dimensionality. For example, if d=12 -> [2, 2, 3], the 12 dimensional
# pure state is decomposed into 3 density matrices, a 2x2, a 2x2, and a 3x3.
# Obviously, there are many different ways to decompose the same pure state,
# depending on the prime factorization and its symmetries. The question is
# how to juxtapose all that information visually. For now, considering as
# we are choosing states at random, there's no reason to prefer one
# decomposition a priori, so we make the canonical choice.
# Returns a list of: the constellation of the pure state, followed by
# the emanations of the mixed states.
def shatter(v):
    d = len(v)
    primes = factorize(d)
    pure = qutip.Qobj(v)
    pure.dims = [primes,[1]*len(primes)]
    souls = [pure.ptrace(i).full() for i in range(len(primes))]
    return constellate(v), [emanate(soul) for soul in souls]

# Returns a randomly generated quantum system:
# a random pure state for the state,
# a random hermitian matrix for the hamiltonian,
# aka the energy operator aka the time evolver.
def bang(n):
    return qutip.rand_ket(n).full().T[0], \
        qutip.rand_herm(n).full()

# Initialize quantum simulation
state, hamiltonian = bang(n)

# Initialize video settings
vpython.scene.width = 1000
vpython.scene.height = 1000
vpython.scene.range = 1.3
# Set our initial vantage point at random.
vpython.scene.forward = vpython.vector(random.random(), random.random(), random.random())

# Create the celestial sphere, then shatter the pure state, capturing its
# light in separate vessels. The stars of the pure state are constellated.
# Then the separate impure spheres, one for each prime dimensional
# piece, are explored. Within each impure sphere, there is a hierarchy of constellations
# given by the spectrum of the density matrix. The stars of each constellation are created,
# their sizes/heights are proportional to the eigenvalue corresponding to the eigenvector
# of the constellation of which they are a part. The radii can't be less than 0.1.
god = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.4)
vessels = shatter(state)
purities, impurities = vessels
pure_stars = [vpython.sphere(pos=vpython.vector(*star), radius=0.1,\
                color=vpython.color.black, opacity=0.6, emissive=True )\
                    for star in purities]
impure_spheres = []
for n, impurity in enumerate(impurities):
    heights, heavens = impurity
    colors = [0, 0, 0]
    levels = []
    for i in range(len(heavens)):
        height = heights[i]
        constellation = heavens[i]
        this_color = [float(n)/float(len(impurities)),\
                     float(len(heavens)-i)/len(heavens),\
                     1]
        impure_stars = []
        for j, star in enumerate(constellation):
            impure_star = vpython.sphere(pos=vpython.vector(*star), radius=0.05+abs(height**2),\
                    color=vpython.color.hsv_to_rgb(vpython.vector(*this_color)), opacity=0.8, emissive=True)
            curve = None
            if j == len(constellation)-1:
                curve = vpython.curve(radius=1, color=vpython.color.red, pos=[vpython.vector(*constellation[j]), vpython.vector(*constellation[0])])
            else:
                curve = vpython.curve(radius=1, color=vpython.color.red, pos=[vpython.vector(*constellation[j]), vpython.vector(*constellation[j+1])])
            curve.visible = True
            impure_stars.append((impure_star, curve))
        levels.append(impure_stars)
    impure_spheres.append(levels)

# Free will interface
# Mouse click callback
def click(event):
    pick = vpython.scene.mouse.pick
    if pick.emissive:
        pick.emissive=False
    else:
        pick.emissive=True
        
vpython.scene.bind('click', click)
    
# Time evolution:
# Exponentiates the hamiltonian, integrating up to the current
# time step, giving a unitary transformation implementing
# continuous time evolution up to now. The unitary time evolution
# operator is applied to the original state to get the next.
# This resulting state is shattered, and the locations of all the stars,
# in their separate constellations and in their separate spheres, impure,
# and pure, are updated, along with their radii.
# Adjust: 
#  dt gives the width of a timestep in the simulation
#  rate(f) sets the frequency of updates in real time
#    to increase the effect of verisimilitude.
t = 0
dt = 0.002
while False:
    vpython.rate(10000)
    time_step = scipy.linalg.expm(-2*math.pi*complex(0,1)*hamiltonian*t)
    tomorrows_vessels = shatter(time_step.dot(state))
    tomorrows_purities, tomorrows_impurities = tomorrows_vessels
    for i in range(len(pure_stars)):
        pure_stars[i].pos = vpython.vector(*tomorrows_purities[i])
    for i in range(len(tomorrows_impurities)):
        tomorrows_heights = tomorrows_impurities[i][0]
        tomorrows_heavens = tomorrows_impurities[i][1]
        for j in range(len(tomorrows_heavens)):
            tomorrows_height = tomorrows_heights[j]
            tomorrows_constellation = tomorrows_heavens[j]
            stars = impure_spheres[i][j]
            for k in range(len(stars)):
                star, line = stars[k][0], stars[k][1]
                stars[k][0].pos = vpython.vector(*tomorrows_constellation[k])
                stars[k][0].radius = 0.05+float(abs(tomorrows_height)**2)
                stars[k][1].clear()
                if k > 0:
                    if k == len(stars)-1:
                        stars[k][1].append([stars[0][0].pos, stars[k][0].pos])
                    else:   
                        stars[k][1].append([stars[k-1][0].pos, stars[k][0].pos])
                else:
                    stars[0][1].append([vpython.vector(*tomorrows_constellation[-1]), stars[k][0].pos])
    t += dt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>